In [2]:
# !! hadoop fs -rm -r /user/root/data/amazon/review_csv
!! hadoop fs -ls -h /user/root/data/amazon/

['Found 11 items',
 '-rw-r--r--   2 root supergroup      87.8 G 2017-10-28 00:54 /user/root/data/amazon/complete.json',
 'drwxr-xr-x   - root supergroup           0 2017-10-31 04:33 /user/root/data/amazon/complete_csv',
 'drwxr-xr-x   - root supergroup           0 2017-10-31 23:42 /user/root/data/amazon/complete_sample',
 '-rw-r--r--   1 root supergroup       3.1 G 2017-10-27 07:29 /user/root/data/amazon/item_dedup.csv',
 '-rw-r--r--   1 root supergroup      54.3 G 2017-10-27 07:34 /user/root/data/amazon/item_dedup.json',
 'drwxr-xr-x   - root supergroup           0 2017-10-31 04:16 /user/root/data/amazon/review_Books',
 'drwxr-xr-x   - root supergroup           0 2017-11-02 09:16 /user/root/data/amazon/review_csv',
 '-rw-r--r--   2 root supergroup      18.4 G 2017-10-30 23:50 /user/root/data/amazon/reviews_Books.json',
 '-rw-r--r--   1 root supergroup      82.9 K 2017-10-27 07:08 /user/root/data/amazon/sample.json',
 '-rw-r--r--   2 root supergroup      54.3 G 2017-10-31 02:00 /user/r

In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#__author__:yanbin
from pyspark import SparkContext,SparkConf,StorageLevel
from operator import add,itemgetter
from pyspark.sql import SparkSession,Row
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import time
import math
class ItemCF_spark(object):  
    def __init__(self,df,item_pair_sim=None,spark=None,topN=10,NN=100):
        self.spark = spark
        if (self.spark is None):
            conf = SparkConf().setAppName("itemCF").setMaster("yarn")
            self.spark = SparkSession.builder\
                .config(conf=conf) \
                .enableHiveSupport()\
                .getOrCreate()
        sc = spark.sparkContext 
        self.df = df
        self.item_pair_sim = item_pair_sim
        self.topN = topN
        self.NN = NN

    def itemSimilarity(self):
        # RDD[(uid,(aid,score))] 
        user_item_score = self.df.rdd.map(lambda x:(x[0],[x[1],x[2]]))
        item_score_pair = user_item_score.join(user_item_score)\
                        .map(lambda x:((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
        item_pair_ALL = item_score_pair.map(lambda f:(f[0], f[1][0] * f[1][1])).reduceByKey(add,300)
        item_pair_XX_YY = item_pair_ALL.filter(lambda f:f[0][0] == f[0][1])
        item_pair_XY = item_pair_ALL.filter(lambda f:f[0][0] != f[0][1])
        #RDD[(aid1,score11 * score11 + score21 * score21)] 
        item_XX_YY = item_pair_XX_YY.map(lambda f:(f[0][0], f[1]))
        #RDD(aid1,((aid1,aid2,XY),XX))
        item_XY_XX = item_pair_XY.map(lambda f:(f[0][0], (f[0][0], f[0][1], f[1]))).join(item_XX_YY) 
        #RDD[(aid2,((aid1,aid2,
        #           score11 * score12 + score21 * score22,score11 * score11 + score21 * score21),
        #           score12 * score12 + score22 * score22))] 
        item_XY_XX_YY = item_XY_XX.map(lambda f:(f[1][0][1],(f[1][0][0],f[1][0][1],f[1][0][2],f[1][1]))).join(item_XX_YY)  
        # item_XY_XX_YY中的(aid1,aid2,XY,XX,YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22,score11 * score11 + score21 * score21,score12 * score12 + score22 * score22)]       
        item_pair_XY_XX_YY = item_XY_XX_YY.map(lambda f:(f[1][0][0], f[1][0][1], f[1][0][2], f[1][0][3], f[1][1]))  
        # item_pair_XY_XX_YY为(aid1,aid2,XY / math.sqrt(XX * YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22 / math.sqrt((score11 * score11 + score21 * score21)*(score12 * score12 + score22 * score22))] 
        item_pair_sim = item_pair_XY_XX_YY.map(lambda f :(f[0], (f[1], f[2] / math.sqrt(f[3] * f[4]))))  
        return item_pair_sim
    

    def train(self):
        item_pair_sim = self.itemSimilarity()
        item_pair_sim.persist(storageLevel=StorageLevel(True, True, False, True, 1))
        self.item_pair_sim=item_pair_sim
    



def recommend(df,item_pair_sim,NN=100,topN =10,Normalization=False):
    def itemNN(item_pair_sim,NN=100,Normalization=False):
        item_sim = item_pair_sim.filter(lambda f:f[1][1]>0.05)\
                            .groupByKey()\
                            .mapValues(list)
        if Normalization:
            def norm(x):
                m =  __builtin__.max([i[1] for i in x])
                l = []
                for i in x:
                    l.append((i[0],i[1]/m))
                return l
            item_sim = item_sim.mapValues(lambda x:norm(x))
        item_simNN = item_sim.mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:NN])\
                            .collectAsMap()
        return item_simNN
    
    def getOrElse(f,item_sim_bd):
        items_sim = item_sim_bd.value.get(f[0][1]) 
        if items_sim is None:
            items_sim = [("0", 0.0)]
        for w in items_sim:
            yield ((f[0][0],w[0]),w[1]*f[1])
            
    user_item_score = df.rdd.map(lambda x:((x[0],x[1]),x[2]))
    item_sim_bd = sc.broadcast(itemNN(item_pair_sim,NN=NN,Normalization=Normalization))
#     /* 
#      * 提取item_sim_user_score为((user,item2),sim * score) 
#      * RDD[(user,item2),sim * score] 
#      */  

    user_item_simscore = user_item_score.flatMap(lambda f:getOrElse(f,item_sim_bd))\
                                        .filter(lambda f:f[1]> 0.03)  
#       /*
#      * 聚合user_item_simscore为 (user,（item2,sim1 * score1 + sim2 * score2）)
#      * 假设user观看过两个item,评分分别为score1和score2，item2是与user观看过的两个item相似的item,相似度分别为sim1，sim2 
#      * RDD[(user,item2),sim1 * score1 + sim2 * score2）)] 
#      */  
    user_item_rank = user_item_simscore.reduceByKey(add,1000)  

#     /* 
#      * 过滤用户已看过的item,并对user_item_rank基于user聚合 
#      * RDD[(user,CompactBuffer((item2,rank2）,(item3,rank3)...))] 
#      */  
    user_items_ranks = user_item_rank.subtractByKey(user_item_score)\
                                     .map(lambda f:(f[0][0], (f[0][1], f[1])))\
                                     .groupByKey()  
#     /* 
#      * 对user_items_ranks基于rank降序排序，并提取topN,其中包括用户已观看过的item 
#      * RDD[(user,ArrayBuffer((item2,rank2）,...,(itemN,rankN)))] 
#      */  
    user_items_ranks_desc = user_items_ranks.mapValues(list)\
                            .mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:topN])
    return user_items_ranks_desc

from Evaluator import Evaluator
def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

if __name__ == "__main__": 
    import os
    PYSPARK_PYTHON = "/usr/bin/python2.7"
    os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
    conf = SparkConf().setAppName("itemCF").setMaster("yarn")
    conf.set('spark.executor.cores','30')
    conf.set('spark.executor.memory','90g')
    conf.set('spark.executor.instances','4')
    conf.set('spark.sql.shuffle.partitions','400')
    conf.set('spark.default.parallelism','200')
    conf.set('spark.scheduler.mode','fair')
    conf.set("spark.shuffle.file.buffer","128k")
    conf.set("spark.reducer.maxSizeInFlight","96M")
    conf.set("spark.yarn.executor.memoryOverhead","2g")
    conf.set("spark.shuffle.memoryFraction" ,"0.1")
    conf.set("spark.yarn.scheduler.heartbeat.interval","7200000ms")
    conf.set("spark.executor.heartbeatInterval","7200000ms")
    conf.set("spark.network.timeout", "7200000ms")
    conf.set("spark.memory.storageFraction","0.25")
    spark = SparkSession.builder\
        .config(conf=conf) \
        .getOrCreate()
    sc=spark.sparkContext 
    start = time.time()
    inputPath = "data/amazon/rating_parquet"
#     inputPath = "data/amazon/choosed_parquet"
    ratingDf = spark.read.parquet(inputPath)
#     ratingDf,_ = ratingDf.randomSplit([1.0,9.0],seed=40)
    ratingDf = ratingDf.repartition(300)
#     ratingDf = ratingDf.select('user','item','rating')
#     user_count = ratingDf.groupBy('user').agg(count('*').alias('user_count')).filter('user_count>100')
#     item_count = ratingDf.groupBy('item').agg(count('*').alias('item_count')).filter('item_count>100')
#     ratingDf = ratingDf.join(user_count,on='user',how='inner')
#     ratingDf = ratingDf.join(item_count,on='item',how='inner')
    ratingDf.printSchema()
    ratingDf.show(5)
    n = ratingDf.count()
#     ratingDf = ratingDf.withColumn('score',col('rating')*0+1).select('user','item','score')    
    print 'total lines: %s' %n
    train,test = ratingDf.randomSplit([4.0,1.0],seed=40)
    train.persist(storageLevel=StorageLevel(True, True, False, True, 1))
    test.persist(storageLevel=StorageLevel(True, True, False, True, 1))
    itemCF = ItemCF_spark(df=train,spark=spark)
    itemCF.train()
    for NN in [5,10,20,40,60,80,100,120]:
        recTopN = recommend(train,itemCF.item_pair_sim,NN=NN,topN=10,Normalization=True)
        pre = spark.createDataFrame(data=recTopN.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1][0],x[1][1])),
                                    schema=['user','item','rating'])
        evaluator = Evaluator(train,test,pre)
        (precise,recall,coverage,popularity) = evaluate(evaluator)
        end = time.time()
        print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
                %(NN,precise*100,recall*100,coverage*100,popularity,end-start))
        pre.write.parquet(partitionBy='user',path='data/amazon/result/itemCF/itcf_'+str(NN),mode='ignore')
        start = time.time()
        

root
 |-- user: string (nullable = true)
 |-- item: string (nullable = true)
 |-- rating: double (nullable = true)

+-------------+----------+------+
|         user|      item|rating|
+-------------+----------+------+
|AFS81Q840AOHC|B00I3MQNWG|   5.0|
|AFT8FTOITPZG5|044021002X|   4.0|
|AFU4E0FKGUQ7A|B00005JLG2|   3.0|
|AFV2584U13XP3|B005NQ5LPK|   5.0|
|  AFVQZQ8PW0L|0345495152|   5.0|
+-------------+----------+------+
only showing top 5 rows

total lines: 142831980


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: ShuffleMapStage 8 (reduceByKey at <ipython-input-1-ab6d2672c546>:29) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: Failed to connect to node3/172.20.128.162:38375
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:361)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:336)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:54)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala

In [2]:
sc.stop()
spark.stop()
# MEMORY_AND_DISK 

In [22]:
import time
start = time.time()
inputPath = "data/amazon/complete_csv"
schema = StructType([
        StructField("user", StringType(), True),
        StructField("item", StringType(), True),
        StructField("rating", DoubleType(), True),
        StructField("text", StringType(), True),
        StructField("summary", StringType(), True)])
ratingDf = spark.read.csv(inputPath,header=True,schema=schema)
# ratingDf = ratingDf.select('user','item','rating')
# ratingDf.printSchema()
# ratingDf.show(5)
# n = ratingDf.count()
# print 'total lines: %s' %n
# ratingDf,_ = ratingDf.randomSplit([1.0,999.0])
# n = ratingDf.count()
# print 'total lines after reduce: %s' %n
# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from pyspark.ml.pipeline import Pipeline,PipelineModel
# from pyspark.ml.feature import StringIndexer
# from pyspark.ml.recommendation import ALS
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import *
# from pyspark.sql.functions import *  
# trainDf,testDf = ratingDf.randomSplit([9.0,1.0])
# userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(ratingDf)
# itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(ratingDf)
# alsModel = ALS(rank=10,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.1)
# pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
# pipeAlsModel = pipeAls.fit(trainDf)
# preTrain = pipeAlsModel.transform(trainDf)
# preTest= pipeAlsModel.transform(testDf)
# preTrain.show(10)
# preTest.show(10)
# end = time.time()
# print 'spend %s s' %(end-start)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
n = ratingDf.count()
ui_count  = ratingDf.agg(count('*').alias('ui'))
user_count = ratingDf.groupBy('user').agg(count('*').alias('user_count'))
item_count = ratingDf.groupBy('item').agg(count('*').alias('item_count'))
count_union = user_count.describe('user_count')\
          .join(item_count.describe('item_count'),on='summary',how='inner')
count_union.show()

In [63]:
evaluator = RegressionEvaluator()\
      .setMetricName("rmse")\
      .setLabelCol("rating")\
      .setPredictionCol("prediction")
rmse = evaluator.evaluate(preTest.select('userid','itemid','rating','prediction').na.drop())
print 'rmse is %s' %rmse

rmse is 4.78507940484


In [69]:
# ratingDf,_ = ratingDf.randomSplit([1.0,99999.0])
# ratingDf.count()
ratingDf.write.csv(path='data/amazon/complete_sample',header=True)

In [23]:
review_csv=ratingDf.select(concat_ws('-',ratingDf.user,ratingDf.item).alias('ui'),
                ratingDf.rating,
                ratingDf.summary,
                ratingDf.text)
review_csv.write.csv(path='data/amazon/review_csv',header=True)